# Analyzing FHIR data in Databricks

In [2]:
# Some initial setting:
storageAccountName = "<storage account>"
storageAccountKey = "<storage key>"

In [3]:
dbutils.fs.mount(
  source = "wasbs://dataexport@" + storageAccountName + ".blob.core.windows.net",
  mount_point = '/mnt/dataexport',
  extra_configs = {"fs.azure.account.key." + storageAccountName + ".blob.core.windows.net":storageAccountKey})

Out[ 2 ]: True

In [4]:
%sql CREATE TEMPORARY TABLE observationTable USING json OPTIONS (path "/mnt/dataexport/Observation.json")

In [5]:
%sql CREATE TEMPORARY TABLE patientTable USING json OPTIONS (path "/mnt/dataexport/Patient.json")

In [6]:
%sql 
CREATE OR REPLACE TEMPORARY VIEW temp_heights AS 
  SELECT * FROM (
    SELECT 
      SUBSTRING_INDEX(subject.reference,'/',-1) AS patient, 
      valueQuantity.value as heightValue, 
      valueQuantity.unit as heightUnit,  
      ROW_NUMBER() OVER (PARTITION BY subject.reference ORDER BY issued DESC) AS rn 
   FROM observationTable WHERE code.coding[0].code = "8302-2") tm 
  WHERE tm.rn = 1

In [7]:
%sql 
CREATE OR REPLACE TEMPORARY VIEW temp_weights AS 
  SELECT * FROM (
    SELECT 
      SUBSTRING_INDEX(subject.reference,'/',-1) AS patient, 
      valueQuantity.value as weightValue, 
      valueQuantity.unit as weightUnit,  
      ROW_NUMBER() OVER (PARTITION BY subject.reference ORDER BY issued DESC) AS rn 
   FROM observationTable WHERE code.coding[0].code = "29463-7") tm 
  WHERE tm.rn = 1

In [8]:
%sql 
CREATE OR REPLACE TEMPORARY VIEW temp_latitude AS 
  SELECT id, coord.valueDecimal AS latitude FROM 
    (SELECT id, explode(address[0].extension[0].extension) as coord FROM patientTable) 
  WHERE coord.url = 'latitude';
  
CREATE OR REPLACE TEMPORARY VIEW temp_longitude AS 
  SELECT id, coord.valueDecimal AS longitude FROM 
    (SELECT id, explode(address[0].extension[0].extension) as coord FROM patientTable) 
  WHERE coord.url = 'longitude'

In [9]:
%sql 
SELECT
  patientTable.id, 
  patientTable.name[0].family AS lastName, 
  temp_longitude.longitude AS longitude, 
  temp_latitude.latitude AS latitude, 
  temp_weights.weightValue, temp_heights.heightValue
FROM patientTable 
  INNER JOIN temp_weights ON temp_weights.patient = patientTable.id 
  INNER JOIN temp_heights ON temp_heights.patient = patientTable.id 
  INNER JOIN temp_latitude ON temp_latitude.id = patientTable.id 
  INNER JOIN temp_longitude ON temp_longitude.id = patientTable.id
LIMIT 10

id,lastName,longitude,latitude,weightValue,heightValue
248121bc-fc90-4425-a0f1-bd0e84b5b5da,Wunsch504,-71.32210699999997,42.638893,60.02599904220306,179.59501395251195
24daf013-0c11-4a2d-9943-857749b7d470,Schroeder447,-70.685828,42.629024,49.84804906546639,151.1135034194228
2787697a-4163-4018-b68d-46e451b21fd5,Bogan287,-71.11868,42.376043,107.39098315688696,161.36633023436602
304c1b1c-ea98-484a-af27-514319294486,Hagenes547,-71.42107,42.383367,7.640234186380895,68.83485710840303
56ce4898-b5b5-4846-845e-56bc6c86d07f,Hammes673,-70.937913,41.661303,92.74247008484545,181.02535806008987
5aa50b8c-16ac-4f22-a3f1-f80d6d7d3833,Lynch190,-71.046214,41.930477,67.2005756743314,174.13388032239087
67a067f4-4b08-4235-962f-2ef3c9c9c17e,Leuschke194,-71.268121,42.246673,35.39357624589232,173.46750593512257
92d3e456-38a6-4592-9f92-9e48619224b2,Walter473,-71.436196,42.307905,47.722789070706106,158.79468995154576
96b0eaed-fa2c-4f1a-801f-85e2a7632134,Runolfsdottir785,-71.578779,42.036642,71.77111936667654,176.9905897221011
973be89f-64b1-4a30-a483-00588fbe77f8,Cummings51,-72.539978,42.115454,69.79025363350438,172.57812359593373
